In [1]:
import tensorflow as tf

In [2]:
tf.__version__

'1.0.1'

In [27]:
tf.InteractiveSession()

# Placeholder

계속 말하지만 TF 프로그램은 두단계로 구성됩니다.
* 1. 그래프 만들고
* 2. 그래프에서 op을 실행하기 위해 session 사용

연산에 필요한 값을 모르고 그래프를 먼저 만들 수 있는 것입니다. 이는 마치 함수 f(x,y) = 2x + y 에서 x,y 값을 모르는 것과 같습니다. 말하자면 x와 y는 실제 값에 대한 placeholder인 것입니다. 

constant vs. Variable vs. placeholder

* constant : op
* Variable : class
* placeholder : op

### Placeholder를 사용하는 이유

데이터는 실행 시에 제공되면 되지, 미리 그래프에 올려둘 필요가 없기 때문입니다. 

placeholder에서는 사전을 이용하여 값을 전달해 줍니다.

In [5]:
a = tf.placeholder(dtype=tf.float32, shape=[3])
b = tf.constant([5,5,5], tf.float32)
c = a + b

with tf.Session() as sess:
    # print sess.run(c) # error : You must feed a value for placeholder tensor 'Placeholder' with dtype float and shape [3]
    print sess.run(c, feed_dict={a:[1,2,3]})

[ 6.  7.  8.]


### shape=[None]

Placeholder를 만들 때 shape=[None] 은 어떤 shape을 가지는 텐서라도 받아줄 수 있다는 의미입니다. 

그래프를 만드는데 편의를 제공해 주지만, 반대로 디버깅을 할 때에는 골치거리가 될 수도 있습니다. 

또한 shape이 어떨까에 대한 추론을 오히려 어렵게 만들수도 있습니다.

### Placeholder는 유효한 op으로 당연히 그래프에도 올라가고 텐서보드에서도 확인할 수 있습니다. 

In [8]:
a = tf.placeholder(dtype=tf.float32, shape=[3])
b = tf.constant([5,5,5], dtype=tf.float32)
c = a + b
with tf.Session() as sess:
    writer = tf.summary.FileWriter('./graphs', sess.graph)
    print sess.run(c, feed_dict={a:[1,2,3]})

[ 6.  7.  8.]


In [12]:
# !tensorboard --logdir='./graphs' --port 9879

### 여러개의 data points를 피드하고 싶은 경우

모든 값들을 보낸 다음에 loop을 이용해 한번에 하나씩 처리하면 됩니다. 

In [14]:
with tf.Session() as sess:
    for value_1 in values:
        print sess.run(c, {value: value_1})

NameError: name 'values' is not defined

### 우리는 (변수이건, 상수이건, 뭐라도) 피드 가능한 텐서라면 모두 feed_dict할 수 있습니다.

placeholder는 무언가가 feed되어야 한다는 것을 나타내는 방법에 불과합니다. 

**꼭 placholder로 선언을 해야만 feed할 수 있는게 아닙니다!!!**

어떤 텐서가 feeable한 지 확인하는 방법은 아래와 같이 tf.Graph.is_feedable(tensor) 를 활용하면 됩니다. 

In [19]:
a = tf.constant(1)
tf.get_default_graph().is_feedable(a)

True

In [20]:
b = tf.Variable(2)
tf.get_default_graph().is_feedable(b)

True

In [22]:
c = tf.placeholder(dtype=tf.float32, shape=None)
tf.get_default_graph().is_feedable(c)

True

아래 2개의 예를 비교해 봅시다.

In [32]:
a = tf.add(2,5)
b = tf.multiply(a,3)
with tf.Session() as sess:
    print sess.run(b) # 아주 일반적인 사용 예

21


In [33]:
a = tf.add(2,5)
b = tf.multiply(a,3)
with tf.Session() as sess:
    print sess.run(b, feed_dict={a:10}) # a는 비록 placeholder로 선언한 것 아니지만 이렇게 feed해 줄 수 있는 것이다. 

30


이러한 방식은 테스트를 위해서도 매우 편리합니다. ^^

# Lazy loading

Lazy loading은 어떤 오브젝트를 만들거나 초기화하는데 있어, 실제로 이 오브젝트가 필요할 때까지 생성 및 초기화를 최대한 미루는 것을 의미합니다. 

아래는 normal loading 입니다.

In [38]:
x = tf.Variable(10, name='x')
y = tf.Variable(20, name='y')
z = tf.add(x, y)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter('./my_graph', sess.graph)
    for _ in range(10):
        print sess.run(z)
    writer.close()

30
30
30
30
30
30
30
30
30
30


In [41]:
tf.get_default_graph().as_graph_def() # 그래프 정의에 "Add" 노드가 한번 추가된다.

node {
  name: "Placeholder"
  op: "Placeholder"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: 3
        }
      }
    }
  }
}
node {
  name: "Const"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_FLOAT
        tensor_shape {
          dim {
            size: 3
          }
        }
        tensor_content: "\000\000\240@\000\000\240@\000\000\240@"
      }
    }
  }
}
node {
  name: "add"
  op: "Add"
  input: "Placeholder"
  input: "Const"
  attr {
    key: "T"
    value {
      type: DT_FLOAT
    }
  }
}
node {
  name: "Placeholder_1"
  op: "Placeholder"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: 3
        }
      }
    }
  }
}
node {
  name: "Const_1"
  op: "Const"
  att

아래는 lazy loading 입니다.

In [44]:
x = tf.Variable(10, name='x')
y = tf.Variable(20, name='y')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter('./my_graph/l2', sess.graph)
    for _ in range(10):
        print sess.run(tf.add(x,y)) # lazy loading
    writer.close()

30
30
30
30
30
30
30
30
30
30


In [43]:
tf.get_default_graph().as_graph_def() # 그래프 정의에 "Add" 노드가 10번 추가된다.

node {
  name: "Placeholder"
  op: "Placeholder"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: 3
        }
      }
    }
  }
}
node {
  name: "Const"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_FLOAT
        tensor_shape {
          dim {
            size: 3
          }
        }
        tensor_content: "\000\000\240@\000\000\240@\000\000\240@"
      }
    }
  }
}
node {
  name: "add"
  op: "Add"
  input: "Placeholder"
  input: "Const"
  attr {
    key: "T"
    value {
      type: DT_FLOAT
    }
  }
}
node {
  name: "Placeholder_1"
  op: "Placeholder"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: 3
        }
      }
    }
  }
}
node {
  name: "Const_1"
  op: "Const"
  att

이처럼 그래프에 여러번 추가되게 되면 그래프에 무리를 주게 됩니다.
* 그래프의 로딩이 느려집니다
* 그래프에서 텐서가 이동해 다니는 것이 비싸집니다

이처럼 lazy loading을 잘못 구현하는 것이 TF에서 버그가 아닌 버그 중에 가장 흔한 케이스입니다. 

해결 방법
* op의 definition을 op의 computing/running 부분과 분리시킵니다.
* 파이썬의 property를 이용해 함수가 오직 첫번째 호출되는 경우에만 로드되도록 보장해 줍니다. 
(http://danijar.com/structuring-your-tensorflow-models

# Debugging

그냥 파이썬의 pdb 활용도 가능하다.

pdb.set_trace()로 break point 생성하고 interactive하게 print sess.run(a) 와 같은 식으로 값 확인할 수 있다. 

In [ ]:
import pdb
a = tf.Variable(1)
b = tf.constant(3)
c = tf.add(a, b)
pdb.set_trace() # pdb.set_trace() # 사용법은 잘 모르겠네 ㅎㅎ
print sess.run(c)

--Return--
> <ipython-input-50-45023e4767ec>(5)<module>()->None
-> pdb.set_trace() # pdb.set_trace() # 사용법은 잘 모르겠네 ㅎㅎ
